# libraries & global vars

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# io job_str

In [2]:
path_job_str = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 4 - career/4.3.1 jobs/"

In [3]:
filein_job_str = "job_str_0_empty.xlsx"

In [4]:
fileout_job_str = "job_str_1_mergeold.xlsx"

In [5]:
job_str = pd.read_excel(path_job_str+filein_job_str,dtype="str")

In [6]:
job_str.shape

(7590, 14)

In [7]:
# job_str[0:5]

# io old parsed & coded data \#1: Esther's data

In [8]:
pathin_old_1 = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/2015 data/2015 data - 1 cleaning/"
filein_old_1 = "Eunhos latest_CareerStringsCleaned.xlsx"

In [9]:
# input esther's parsed & coded data
esther = pd.read_excel(pathin_old_1+filein_old_1,dtype="str")

In [10]:
esther.shape

(2210, 16)

In [11]:
# esther[0:10]

In [12]:
# isolate unique strings - I don't want to deal with strings containing multiple jobs
#esther[0:5]
esther.drop_duplicates(subset=["CareerStringID"],keep=False,inplace=True)

In [13]:
esther.shape

(2020, 16)

In [14]:
# esther[0:10]

In [15]:
# job_str
job_str_columns = job_str.columns

In [16]:
job_str_merge = job_str.merge(esther,on="CareerString",how="left",suffixes=('', '_y'))

In [17]:
# confirm that job_str_merge rows are the same as job_str
job_str_merge.shape

(7590, 29)

In [18]:
job_str.shape

(7590, 14)

### impute position & last org from esther

In [19]:
# double-check PositionString is null
# job_str_merge[job_str_merge.PositionString.notnull()]

In [20]:
# impute PositionString with esther.Pos
job_str_merge["PositionString"]=job_str_merge["Pos"]

In [21]:
# double-check OrgString is null
job_str_merge[job_str_merge.OrgString.notnull()]

,CareerStringID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,...,Org4,Pos,Resigned,Graduated,SchoolCareer,PartySchoolTrue,LastSchoolGraduatedFinal,DeathTrue,MarriedTrue,SchoolTrue


In [22]:
# impute OrgString as esther's final organization
job_str_merge["OrgString"]= job_str_merge[["Org1","Org2","Org3","Org4"]].apply(lambda x: " ".join(x.dropna()).strip(), axis=1)

In [23]:
# replace white space with Null
job_str_merge.loc[job_str_merge["OrgString"]=="","OrgString"]=np.nan

In [24]:
# inspect OrgString
# job_str_merge[["OrgString","Org1","Org2","Org3","Org4"]]

In [25]:
# how many OrgStrings did we impute?
job_str_merge[job_str_merge.OrgString.notnull()].shape

(503, 29)

In [26]:
# how many OrgStrings are still null?
job_str_merge[job_str_merge.OrgString.isnull()].shape

(7087, 29)

In [27]:
# how many PositionStrings did we impute?
job_str_merge[job_str_merge.PositionString.notnull()].shape

(1220, 29)

In [28]:
# how many PositionStrings are still null?
job_str_merge[job_str_merge.PositionString.isnull()].shape

(6370, 29)

In [29]:
# update job_str with merged values
job_str = job_str_merge.loc[:,job_str_merge.columns.isin(job_str.columns)]

# io old parsed & coded data \#2: old data

In [30]:
pathin_old_2 = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/2015 data/2015 data - 0 raw/"
filein_old_2 = "leadercareerlinkmou.csv"

In [31]:
old_columns = ["MOUID", "KoreanName", "CareerStringID", "CareerString", "CareerStartYear", "CareerStartMonth", "EventType", "JointAppointment", "Org1", "Org2", "Org3", "Org4", "Org5", "Position", "OrgLevel", "PositionLevel", "OrgPositionString"]

In [32]:
old = pd.read_csv(pathin_old_2+filein_old_2,sep=";",header=0,names=old_columns)

In [33]:
esther.columns

Index(['CareerStringID', 'CareerString', 'CareerStringCleaned', 'Org1', 'Org2',
       'Org3', 'Org4', 'Pos', 'Resigned', 'Graduated', 'SchoolCareer',
       'PartySchoolTrue', 'LastSchoolGraduatedFinal', 'DeathTrue',
       'MarriedTrue', 'SchoolTrue'],
      dtype='object')

In [34]:
# old[0:5]

In [35]:
# select the columns from old that I care about
old_columns_keep = ["CareerString","CareerStartYear","CareerStartMonth","Org1","Org2","Org3","Org4","Org5","Position"]
old = old.loc[:,old.columns.isin(old_columns_keep)]
old.drop_duplicates(inplace=True)
old.shape

In [62]:
# multiple coded strings are causing trouble,
# just as they did in esther's
# drop all duplicate career strings - and deal with multiples again
old = old.drop_duplicates(["CareerString"],keep=False)
old.shape

In [37]:
# check if some CareerStrings in old, not in job_str

In [66]:
# merge old into job_str
job_str_merge = job_str.merge(old,on="CareerString",how="left",suffixes=('', '_y'))

In [67]:
# check that job_str dimensions are the same as before = (7590)
job_str_merge.shape

(7590, 22)

In [68]:
#
job_str.shape

(7590, 14)

### impute old Position

In [69]:
# inspect existing values of PositionString
# job_str_merge[job_str_merge.PositionString.notnull()]

In [70]:
# before we impute PositionString with old.Position, confirm no overlapping data 
job_str_merge[((job_str_merge.PositionString.notnull()) & (job_str_merge.Position.notnull()))]

,CareerStringID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,...,PositionID,PositionName,CareerStartYear_y,CareerStartMonth_y,Org1,Org2,Org3,Org4,Org5,Position


In [71]:
# where PositionString is null, impute PositionString with old.Position 
job_str_merge.loc[job_str_merge.PositionString.isnull(),"PositionString"]=job_str_merge.loc[job_str_merge.PositionString.isnull(),"Position"]

In [72]:
# how many PositionStrings did we impute from esther?
job_str_merge[job_str_merge.PositionString.notnull() & job_str_merge.Position.isnull()].shape

(1220, 22)

In [73]:
# how many PositionStrings did we impute from old?
job_str_merge[job_str_merge.PositionString.notnull() & job_str_merge.Position.notnull()].shape

(1191, 22)

In [74]:
# how many PositionStrings did we impute overall?
job_str_merge[job_str_merge.PositionString.notnull()].shape

(2411, 22)

In [75]:
# how many PositionStrings are still null?
job_str_merge[job_str_merge.PositionString.isnull()].shape

(5179, 22)

### impute old Org

In [76]:
# inspect existing values of OrgString
# job_str_merge[job_str_merge.OrgString.notnull()]

In [77]:
# create OldOrgString as old's final organization
job_str_merge["OldOrgString"]= job_str_merge[["Org1","Org2","Org3","Org4"]].apply(lambda x: " ".join(x.dropna()).strip(), axis=1)

In [78]:
# replace white space with Null
job_str_merge.loc[job_str_merge["OldOrgString"]=="","OldOrgString"]=np.nan

In [79]:
# double-check OrgString
# job_str_merge[["OldOrgString","Org1","Org2","Org3","Org4"]]

In [80]:
# before we impute OrgString with OldOrgString, confirm no overlapping data
print("OrgString:\t",job_str_merge[job_str_merge.OrgString.notnull()].shape)
print("OldOrgString:\t",job_str_merge[job_str_merge.OldOrgString.notnull()].shape)
print("overlapping:\t",job_str_merge[((job_str_merge.OrgString.notnull()) & (job_str_merge.OldOrgString.notnull()))].shape)

OrgString:	 (503, 23)
OldOrgString:	 (1191, 23)
overlapping:	 (0, 23)


In [81]:
# where OrgString is null, impute OrgString with OrgString 
job_str_merge.loc[job_str_merge.OrgString.isnull(),"OrgString"]=job_str_merge.loc[job_str_merge.OrgString.isnull(),"OldOrgString"]

In [82]:
# how many OrgString did we impute from esther?
job_str_merge[job_str_merge.OrgString.notnull() & job_str_merge.OldOrgString.isnull()].shape

(503, 23)

In [83]:
# how many OrgString did we impute from old?
job_str_merge[job_str_merge.OrgString.notnull() & job_str_merge.OldOrgString.notnull()].shape

(1191, 23)

In [84]:
# how many OrgString did we impute overall?
job_str_merge[job_str_merge.OrgString.notnull()].shape

(1694, 23)

In [85]:
# how many OrgString are still null?
job_str_merge[job_str_merge.OrgString.isnull()].shape

(5896, 23)

### impute CareerStartYear and CareerStartMonth

In [86]:
# confirm CareerStartYear and CareerStartMonth are null
job_str_merge[job_str_merge.CareerStartYear.notnull()]

,CareerStringID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,...,PositionName,CareerStartYear_y,CareerStartMonth_y,Org1,Org2,Org3,Org4,Org5,Position,OldOrgString


In [87]:
job_str_merge[job_str_merge.CareerStartMonth.notnull()]

,CareerStringID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,...,PositionName,CareerStartYear_y,CareerStartMonth_y,Org1,Org2,Org3,Org4,Org5,Position,OldOrgString


In [ ]:
# impute CareerStartYear and CareerStartMonth with CareerStartYear_y and CareerStartMonth_y from old
job_str_merge["CareerStartYear"] = job_str_merge["CareerStartYear_y"]
job_str_merge["CareerStartMonth"] = job_str_merge["CareerStartYear_y"]

### update job_str

In [281]:
# update job_str with merged values
job_str = job_str_merge.loc[:,job_str_merge.columns.isin(job_str.columns)]

In [282]:
# check that job_str dimensions are the same as before (7590,14)?
job_str.shape

(7901, 14)

In [ ]:
# why is job_str.shape not 750

# io old parsed & coded data \#3: ?

In [37]:
pathin_old_3 = ""
filein_old_3 = ""

# output jobs_str - now merged with old parsed, coded data

In [44]:
job_str.to_excel(path_job_str+fileout_job_str,index=False,encoding="utf-8")